In [1]:
import pulp

In [2]:
list1=["236","237","161","141","142","239","170","48","162","186"]
list2=[118813,111344,79534,75650,73180,72762,72590,70347,64410,63090]

list3=dict(zip(list1,list2))
list3

{'236': 118813,
 '237': 111344,
 '161': 79534,
 '141': 75650,
 '142': 73180,
 '239': 72762,
 '170': 72590,
 '48': 70347,
 '162': 64410,
 '186': 63090}

In [3]:
# Creates a list of all the supply nodes
drop_off_loc = ["236","237","161","141","142","239","170","48","162","186"]

# Creates a dictionary for the number of units of supply for each supply node
supply = {'236': 118813,
 '237': 111344,
 '161': 79534,
 '141': 75650,
 '142': 73180,
 '239': 72762,
 '170': 72590,
 '48': 70347,
 '162': 64410,
 '186': 63090}

# Creates a list of all demand nodes
pick_up_loc = ["236","237","161","141","142","239","170","48","162","186"]

# Creates a dictionary for the number of units of demand for each demand node
demand = {'236': 118813,
 '237': 111344,
 '161': 79534,
 '141': 75650,
 '142': 73180,
 '239': 72762,
 '170': 72590,
 '48': 70347,
 '162': 64410,
 '186': 63090}

# Creates a list of costs of each transportation path
costs = [
[10.63803066, 11.60614686, 15.32220056, 11.58488895, 15.60665517, 13.53822612, 17.65766042, 18.25891393, 14.71125601, 21.41376282], 
[12.25793377, 10.70192695, 12.00947351, 10.64983507, 12.92001991, 14.82567551, 14.11869359, 14.64590376, 11.61193707, 17.8319642], 
[15.65745826, 11.99720843, 12.51374635, 14.03258814, 13.24577141, 16.57040217, 11.41645603, 11.40782298, 10.65400556, 12.69033026], 
[12.68565285, 10.77172876, 13.87497968, 10.85895231, 14.83448518, 15.78412371, 14.51544294, 16.4086892, 12.09701445, 19.37492683], 
[15.30196748, 13.1039766, 13.72346687, 14.76262458, 11.71369048, 10.88490997, 17.41598234, 11.12446851, 15.31533645, 15.55810569], 
[13.23094703, 14.92030015, 16.9562656, 15.79671217, 11.24735203, 11.0914402, 20.75459941, 13.84435615, 18.29360367, 18.93228007], 
[18.68837186, 14.18869044, 11.66308196, 15.05687923, 17.22359386, 20.65965478, 12.24159947, 14.1681974, 11.1302462, 12.5611252], 
[18.87895062, 15.43952351, 12.46372956, 17.23799366, 11.05453381, 13.88874391, 14.42156923, 12.13210383, 13.54352667, 12.05152167], 
[15.21639799, 11.56684995, 10.65486794, 12.78366744, 14.45222603, 18.0359707, 11.1975332, 13.13097081, 12.46571951, 14.67656966], 
[21.94766206, 17.83676587, 13.05696618, 19.68705189, 15.21865934, 19.00171233, 12.40842086, 12.05752976, 14.23897089, 16.93007778]
]

# The cost data is made into a dictionary
costs = pulp.makeDict([drop_off_loc, pick_up_loc], costs, 0)

costs

defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
            {'236': defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
                         {'236': 10.63803066,
                          '237': 11.60614686,
                          '161': 15.32220056,
                          '141': 11.58488895,
                          '142': 15.60665517,
                          '239': 13.53822612,
                          '170': 17.65766042,
                          '48': 18.25891393,
                          '162': 14.71125601,
                          '186': 21.41376282}),
             '237': defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
                         {'236': 12.25793377,
                          '237': 10.70192695,
                          '161': 12.00947351,
                          '141': 10.64983507,
                          '142': 12.92001991,
                          '239': 14.82567551,
             

In [4]:
# Import PuLP modeler functions
from pulp import *

# Creates the 'prob' variable to contain the problem data
prob = LpProblem("Material Supply Problem", LpMaximize)

C:\Users\risha\AppData\Local\Programs\Python\Python310\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [5]:
# Creates a list of tuples containing all the possible routes for transport
Routes = [(w, b) for w in drop_off_loc for b in pick_up_loc]

# A dictionary called 'Vars' is created to contain the referenced variables(the routes)
vars = LpVariable.dicts("Route", (drop_off_loc, pick_up_loc), 0, None, LpInteger)

In [6]:
# The minimum objective function is added to 'prob' first
prob += (
    lpSum([vars[w][b] * costs[w][b] for (w, b) in Routes]),
    "Sum_of_Transporting_Costs",
)

In [7]:
# The supply maximum constraints are added to prob for each supply node (drop_off_loc)
for w in drop_off_loc:
    prob += (
        lpSum([vars[w][b] for b in pick_up_loc]) <= supply[w],
        "Sum_of_Products_out_of_drop_off_loc_%s" % w,
    )

# The demand minimum constraints are added to prob for each demand node (project)
for b in pick_up_loc:
    prob += (
        lpSum([vars[w][b] for w in drop_off_loc]) >= demand[b],
        "Sum_of_Products_into_pick_up_loc%s" % b,
    )

In [8]:
# The problem is solved using PuLP's choice of Solver
prob.solve()

# Print the variables optimized value
for v in prob.variables():
    if v.varValue!=0:
        print(v.name, "=", v.varValue)
    
# The optimised objective function value is printed to the screen
print("Value of Objective Function = ", value(prob.objective))



Route_141_142 = 64665.0
Route_141_161 = 10985.0
Route_142_162 = 64238.0
Route_142_237 = 8942.0
Route_161_141 = 75650.0
Route_161_237 = 3884.0
Route_162_239 = 64410.0
Route_170_142 = 8515.0
Route_170_236 = 55723.0
Route_170_239 = 8352.0
Route_186_236 = 63090.0
Route_236_186 = 48466.0
Route_236_48 = 70347.0
Route_237_161 = 68549.0
Route_237_186 = 14624.0
Route_237_237 = 28171.0
Route_239_162 = 172.0
Route_239_170 = 72590.0
Route_48_237 = 70347.0
Value of Objective Function =  13531451.09029774
